# Data science 101
This problem and dataset are taken from a paper that studied how choices made while analyzing data by different data science teams could incluence the results: 

[Many analysts, one dataset: Making transparent how variations in analytical choices affect results](https://osf.io/gvm2z/)

This particular approach is in part based on the tutorial _Exploratory Data Analysis in Python_, presented at [PyCon2017](https://github.com/cmawer/pycon-2017-eda-tutorial) which is considerably more thorough (if you are interested)

## Overview
We’re going to explore a dataset describing the players who played in the 2012-13 European football (soccer) professional leagues. Data about the players’ ages, heights, weights, position, skintone rating, and more were included. Focus on exploring the data to find actionable insights, with an overarching goal of answering the following question: **“Are soccer referees more likely to give red cards to dark skin toned players than light skin toned players?”**

## About the data
The dataset is a detailed list of 146,028 dyads (referee-player pairs) including details from the players, referee's, etc. The readme on the website details tyhe data:
```
From a company for sports statistics, we obtained data and profile photos from all soccer players (N = 2,053) playing in the first male divisions of England, Germany, France and Spain in the 2012-2013 season and all referees (N = 3,147) that these players played under in their professional career (see Figure 1). We created a dataset of player–referee dyads including the number of matches players and referees encountered each other and our dependent variable, the number of red cards given to a player by a particular referee throughout all matches the two encountered each other.
```
![](dataDescription.png)

## Analysis overview and general tips
- Before plotting/joining/doing something, have a question or hypothesis that you want to investigate
- Next it is a good idea to get a general sense for what the data looks like
- Draw a plot of what you want to see on paper to sketch the idea
- Write it down, then make the plan on how to get there
- How do you know you aren't fooling yourself
- What else can I check if this is actually true?
- What evidence could there be that it's wrong?


## Load Data
The csv file included with the material includes ~150k player/referee pairs (or 'dyads' as the documentation refers to them).  Memory usage is moderate, at just over 30 MB so we won't worry too much about memory management at present.  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 1) What does the data look like?

In [ ]:
df = pd.read_csv('redcard.csv')
print(df.shape)
print(df.columns)
#data_frame.info()

In [ ]:
df.dtypes

In [ ]:
df.describe().T

## Understand how the data's organized
The dataset is a single file with every interaction between **referee** and **player** as a single row. In other words: Referee A refereed Player B in, say, 10 games, and gave 2 redcards during those 10 games. Then there would be a unique row in the dataset that said:

Referee A, Player B, 2 redcards, ... 

This has several implications that make this first step to understanding and dealing with this data a bit tricky. First, is that the information about Player B is repeated each time -- meaning:

### If we did a simple average of some metric of we would likely get a misleading result by double counting.

For example, asking "what is the average weight of the players?"

In [ ]:
df.height.mean()

In [ ]:
np.mean(df.groupby('playerShort').height.mean())

## Validating data health
It is a good idea to quickly explore the dataset to understand the overall health. We want to gauge:
- Amount of missing data
- Class imbalances
- Duplicates/repetition
- etc.

As a first measure of the data quality we check for empty fields in entries

In [ ]:
null_vals = pd.concat([df.isnull().sum(),100*df.isnull().sum()/df.shape[0]],axis=1)
null_vals.columns = ['n_missing_vals','perc_missing']
null_vals

There are several missing entries in the different categories. In particular, there is missing information on the players skin color, the attribute we want to study.  We can also visualize the distribution of missing data

In [ ]:
sns.heatmap(df.isnull(), cbar=False)

## Exploring the data: skin color

We see that roughly 15% of players are missing information on their skin color from one or two of the raters. These should be excluded if we want to explore the correlation between these and the number of red cards.  Lets check the instances where we only have one rater value

In [ ]:
players  = df.groupby('playerShort')
print('All players: ', len(players), ' and ', len(df), ' entries')
print('Missing rater1 score: ', len(df[df.rater1.isnull()].groupby('playerShort')))
print('Missing rater2 score: ', len(df[df.rater2.isnull()].groupby('playerShort')))

In [ ]:
one_skin_tone_rating = np.logical_xor(df.rater1.isnull(),df.rater2.isnull())
print('Missing one, but not the other: ', one_skin_tone_rating.sum())

## Let's fix the data frame
Good news! Entries have two or no entries for skin color. Let's remove these entries and understand the relationship between the raters. We can check the correlation between the two raters scores

In [ ]:
df = df[df.rater1.notnull()]
sns.heatmap(df.isnull(), cbar=False)

## Difference between the two raters

In [ ]:
# plot the rater scores in heatmap
fig, ax = plt.subplots(figsize=(12, 8))
sns.heatmap(pd.crosstab(df.rater1, df.rater2), cmap='Blues', annot=True, fmt='d', ax=ax)
ax.set_title("Correlation between Rater 1 and Rater 2\n")
fig.tight_layout()
# make table with correlation as well
(df[['rater1','rater2']].corr()**2).T

### More good news! 
The two skin tone scores are highly correlated. We can start by engineering our first feature and simply take the average of both scores and take a look at the distribution

In [ ]:
df['skin'] = df[['rater1','rater2']].mean(axis=1)
df.head()

### What do the skintone rating look like? European teams are mostly white

In [ ]:
skintone = df.groupby('playerShort').skin.mean()
sns.distplot(skintone)

## Other player correlations
Just to illustrate lets take a look at other pairwise relationships in player attributes. Lets create a second dataframe with only the players information

In [ ]:
player_index = 'playerShort'
player_cols = [#'player', # drop player name, we have unique identifier
               'birthday',
               'height',
               'weight',
               'position',
               'photoID',
                'skin'
              ]
players = df.groupby('playerShort').agg({col:'max' for col in player_cols})
players.head().T

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
pd.tools.plotting.scatter_matrix(players[['height', 'weight', 'skin']], alpha=0.2, diagonal='hist', ax=ax)

## Quick detour: weight vs. height
As expected, there is a clear correlation between the weight and height of players. Lets focus on this specific pair to extract a relationship.

In [ ]:
from sklearn.linear_model import LinearRegression
players1 = players[players.height.notnull()]
players1 = players1[players1.weight.notnull()]

X = np.array(players1.weight).reshape(-1,1)
y = np.array(players1.height).reshape(-1,1)
reg = LinearRegression().fit(X,y)
print(reg.coef_, reg.intercept_)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.regplot('weight', 'height', data=players, ax=ax)
ax.set_ylabel("Height [cm]")
ax.set_xlabel("Weight [kg]")
fig.tight_layout()
fig.text(0.1,0.8, 'y = %0.2lf x + %0.2lf' %(reg.coef_ , reg.intercept_), fontsize = 20)


## Positions
It is possible that the player position has a strong influece on the number of cards as well.

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))
players.position.value_counts(dropna=False).plot(kind='barh', ax=ax)
ax.set_xlabel('Position')

## Classify positions
In order to use this information later, lets split the positions into four categories. Modify th original datframe that has 

In [ ]:
defense = ['Center Back','Defensive Midfielder', 'Left Fullback', 'Right Fullback', ]
midfield = ['Right Midfielder', 'Center Midfielder', 'Left Midfielder',]
forward = ['Attacking Midfielder', 'Left Winger', 'Right Winger', 'Center Forward']
keeper = 'Goalkeeper'

# modifying dataframe -- adding the aggregated position categorical position_agg
df.loc[df['position'].isin(defense), 'position_agg'] = "Defense"
df.loc[df['position'].isin(midfield), 'position_agg'] = "Midfield"
df.loc[df['position'].isin(forward), 'position_agg'] = "Forward"
df.loc[df['position'].eq(keeper), 'position_agg'] = "Keeper"

In [ ]:
player_index = 'playerShort'
player_cols = ['birthday',
               'height',
               'weight',
               'position_agg',
               'redCards',
                'skin'
              ]
players = df.groupby('playerShort').agg({col:'max' for col in player_cols})
players.groupby('position_agg').redCards.mean().T

No statistically significant difference, though defense is more likely to have red cards...surprising that Keepers have a decent amount

## Exploring the data: refs and red cards
Most games have few red cards, i.e. the data set is very unbalanced, especially for cases where a player receives two.  We also have a second scenario where a player receives a red card due to multiple yellow cards. Let't collapse these into a single field. At present we can look at the average number of red cards each player receives per game

In [ ]:
df['total_reds'] = (df.redCards + df.yellowReds)/df.games
plt.hist(df.groupby('playerShort').total_reds.mean())

In [ ]:
df.head

In [ ]:
df.redCards.value_counts()

In [ ]:
# Almost all of the players appear with more than one referee
print('% of players with multiple refs: {:.4f}'.format(sum(df.playerShort.duplicated())/df.shape[0]))
# The same (but to a slightly lesser degree) is true for refs
print('% of refs with multiple players: {:.4f}'.format(sum(df.refNum.duplicated())/df.shape[0]))

In [ ]:
# Store categorical data as the appropriate data type
categorical_features = ['playerShort','club','leagueCountry','position','refNum','refCountry']
df[categorical_features] = df[categorical_features].astype('category')

## Feature selection/engineering
Generally speaking we would like to reduce the number of features to the bare minimum needed to explain the data.

First of all, as we saw, the skin tone scores from both raters are hgihly correlated so we can average them. We make a copy of the data frame so we don't breat anything

We would like to drop the full names of players as this provides little information of use, we should first verify that that the player short names are unique 

In [ ]:
# Confirm that there is a 1:1 mapping from short to full name before dropping full name
flag_non_unique = False
for player_short in df.playerShort.unique():
    if df.player[df.playerShort==player_short].nunique() != 1:
        print('{} maps to multiple values'.format(player_short))
        flag_non_unique = True
if not flag_non_unique:
    print('1:1 mapping')


In [ ]:
df.head()

Select features to drop

In [ ]:
features_to_drop = ['player','birthday','photoID','yellowCards','rater1','rater2','yellowReds','redCards','games',\
                   'Alpha_3','nIAT','seIAT','nExp','seExp','playerShort','refCountry']
# features_to_drop = ['player','birthday','photoID','rater1','rater2','games',\
#                    'Alpha_3','nIAT','seIAT','nExp','seExp','playerShort','refCountry']
df_ = df.drop(features_to_drop,axis=1)

## Quick QA
Just check the shape of returned data frame and some histograms to verify that the data is still good

In [ ]:
df_.shape
df_.head()

In [ ]:
df_.describe()

In [ ]:
# Check distribution of numerical variables
_ = df_.select_dtypes(['float64', 'int64']).hist(figsize=(10,10))

This all looks fine. Even the weight, with a max of exactly 100 kg, looks right

In [ ]:
(df_.corr()**2).T


At first glance it doesn't look like skin tone correlated very heavily with anything...

In [ ]:
# Get rid of unused categories
categorical_features = df_.select_dtypes('category').columns
df_[categorical_features] = df_[categorical_features].apply(lambda x: x.cat.remove_unused_categories())

In [ ]:
df_.select_dtypes('category').nunique()

## Modelling
Now let's see if we can use machine learning to predict establish weather or not the color of a players skin will influence the likelihood of receiving a red card.  For this we will use the readom forest implementation included in the python package [scikit-learn](http://scikit-learn.org/stable/)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics

First we drop the entries with missing values, and drop the information regarding the ref and sprots club. We make a copy since we might want to evaluate the fairness of referees in the future.

In [ ]:
df_rdm_frst = df_.dropna()
df_rdm_frst = df_rdm_frst.drop(['refNum','club'],axis=1)

In [ ]:
df_rdm_frst.select_dtypes('category').nunique()

In [ ]:
df_rdm_frst.head()

Now we input prepare the inputs and labels for the model.  The number of total red cards per game will be used as our label (i.e. feature we want to predict), the rest of the features will be used as inputs.

In [ ]:
X = df_rdm_frst.drop('total_reds',axis=1)
y = df_rdm_frst.total_reds

Since sklearn can't handle categoricals natively, we turn them into dummy variables, as an example for how this works:

In [ ]:
s = pd.Series(list('abca'))
print(s)
print(pd.get_dummies(s))

In [ ]:
X_encoded = pd.get_dummies(X)

## Grid search
A random forest aglorith has several hyperaparemeters that can be tweaked to improve its performance, for instance the number of estimators and the maximum number of features to be used in each. We set up a grid search to find the optimum case for out scenario

In [ ]:
param_grid = {'n_estimators': (5,10,50,100),
              'max_features': (5,8, 12, 16)
             }
clf = RandomForestRegressor()
grid = RandomizedSearchCV(clf, param_grid, cv=4, scoring='r2')

In [ ]:
grid.fit(X_encoded,y)

The best performing scenario:

In [ ]:
grid.best_score_
grid.best_params_

## Performance
We can now check the algorithms performance at predicting the player's red cards given these features according to some norm or score. In this case we use the r2 score where a perfect case is a score of 1 and the worst case is 0.

In [ ]:
y_hat = grid.predict(X_encoded)
r2_original = metrics.r2_score(y,y_hat)
print('Score including the skin tone as a feature: ', r2_original)

## Now ignore skin tone
We follow the same procedure but now remove the skin tone as a factor and again score the performance at trying to predict how likely a player is to receive red cards

In [ ]:
tmpx = X_encoded.skin.values
np.random.shuffle(tmpx)
X_perm = X_encoded.copy()
X_perm.skin = tmpx

y_hat_perm = grid.predict(X_perm)
r2_perm = metrics.r2_score(y,y_hat_perm)
r2_perm

## What next?
The score drops substantially! This is a strong indication that skin tone is playin a role in determining the likelyhood of a player receiving red cards.  There are many other factor we could hope to explore:
- Does the referee country/nationality have an effect?
- How about including the player position (which we engineered)?
- Is there a specific bias towards certain sports clubs? 
- etc.
